In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy.stats import uniform
from pathlib import Path

In [ ]:
def load_data(filename, gwas_dir, sep):
    """Load GWAS data, filter P values, and return a DataFrame."""
    filepath = os.path.join(gwas_dir, filename)
    data = pd.read_csv(filepath, sep=sep)
    
    # Print out number of imputed and original variants by checking column "imputation_status"
    if "imputation_status" in data.columns:
        print(f"Number of imputed variants: {len(data[data['imputation_status'] == 'imputed'])}")
        print(f"Number of original variants: {len(data[data['imputation_status'] == 'original'])}")
    
    # Print rows where pvalue is 0
    zero_pvals = data[data["pvalue"] == 0]
    if len(zero_pvals) > 0:
        print(f"\nFound {len(zero_pvals)} variants with p-value = 0:")
        print(display(zero_pvals))
    
    # Filter out p-values that are 0 or > 1
    data = data[(data["pvalue"] > 0) & (data["pvalue"] <= 1)]
    return data

In [ ]:
def plot_manhattan(gwas, chr_col, bp_col, p_col, snp_col, title):
    """Generate a Manhattan plot."""
    gwas["-log10P"] = -np.log10(gwas[p_col])
    plt.figure(figsize=(20, 10))
    gwas[chr_col] = gwas[chr_col].astype("category")
    gwas.sort_values(by=[chr_col, bp_col], inplace=True)
    gwas["ind"] = range(len(gwas))
    groups = gwas.groupby(chr_col)

    for i, (name, group) in enumerate(groups):
        plt.scatter(
            group["ind"],
            group["-log10P"],
            s=5,
            label=name,
            color=plt.cm.tab20.colors[i % 20]
        )

    plt.axhline(-np.log10(5e-8), color="red", linestyle="--", label="Genome-wide line")
    plt.xlabel("Chromosome")
    plt.ylabel("-log10(p-value)")
    plt.title(title)
    plt.legend(title="Chromosome", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.ylim(0, 10)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_qq(p_values, title, data_df=None, log_p_threshold=10):    
    """
    Generate a QQ plot for GWAS p-values.
    
    Parameters:
    -----------
    p_values : array-like
        Array of p-values to plot
    title : str
        Title for the plot
    data_df : pandas.DataFrame, optional
        DataFrame containing the full GWAS data
    log_p_threshold : float, default=10
        Threshold for -log10(p) value to identify significant variants
    """
    # Calculate expected p-values (under uniform distribution)
    n = len(p_values)
    expected_p_values = np.arange(1, n + 1) / (n + 1)
    
    # Convert to -log10 scale
    observed_log_p = -np.log10(p_values)
    expected_log_p = -np.log10(expected_p_values)
    
    # Print and filter out significant variants
    if data_df is not None:
        significant_mask = observed_log_p > log_p_threshold
        significant_variants = data_df[significant_mask]
        if len(significant_variants) > 0:
            print(f"Found {len(significant_variants)} variants with -log10(p) > {log_p_threshold} for {title}:")
            print(display(significant_variants))
        
        # Filter out significant variants from the plot
        observed_log_p = observed_log_p[~significant_mask]
        expected_log_p = expected_log_p[~significant_mask]
    
    qqplot_2samples(expected_log_p, observed_log_p, xlabel="Expected -log10(p)", ylabel="Observed -log10(p)", line='45')
    plt.title(f"{title}\n(excluding points with -log10(p) > {log_p_threshold})")
    plt.show()

In [ ]:
def analyze_random_phenotypes(num_phenotypes, gwas_dir, filename_suffix):
    """
    Analyze multiple random phenotypes by generating Manhattan and QQ plots.
    
    Parameters:
    -----------
    num_phenotypes : int
        Number of random phenotypes to analyze
    gwas_dir : Path or str
        Directory containing the GWAS files
    filename_suffix : str
        Suffix of the filename
    """
    for phenotype in range(num_phenotypes):
        filename = f"random.pheno{phenotype}.{filename_suffix}"
        print(f"Using gwas directory: {gwas_dir} with filename suffix: {filename_suffix}...")
        # Load data
        gwas = load_data(filename, gwas_dir)
        print(f"Phenotype {phenotype}: Dimensions = {gwas.shape}. First 5 rows:")
        print(display(gwas.head()))
        
        # Manhattan plot
        plot_manhattan(
            gwas,
            chr_col="chromosome",
            bp_col="position",
            p_col="pvalue",
            snp_col="variant_id",
            title=f"Manhattan plot for phenotype {phenotype}"
        )
        
        # QQ plot
        plot_qq(gwas["pvalue"], title=f"QQ plot for phenotype {phenotype}", data_df=gwas)
        print("\n")

In [ ]:
def analyze_random_phenotypes_twas(num_phenotypes, twas_dir, twas_name, filename_suffix, sep):
    """
    Analyze multiple random phenotypes by generating Manhattan and QQ plots.
    
    Parameters:
    -----------
    num_phenotypes : int
        Number of random phenotypes to analyze
    gwas_dir : Path or str
        Directory containing the GWAS files
    filename_suffix : str
        Suffix of the filename
    """
    for phenotype in range(num_phenotypes):
        filename = f"random.pheno{phenotype}{filename_suffix}"
        print(f"Using twas directory: {twas_dir} with filename suffix: {filename_suffix}...")
        # Load data
        twas = load_data(filename, twas_dir, sep=sep)
        print(f"Phenotype {phenotype}: Dimensions = {twas.shape}. First 5 rows:")
        print(display(twas.head()))
        
        # QQ plot
        plot_qq(twas["pvalue"], title=f"QQ plot for phenotype {phenotype} using {twas_name}", data_df=twas)
        print("\n")

def analyze_random_phenotypes_twas_spredixcan(num_phenotypes, twas_dir, twas_name, filename_suffix):
    analyze_random_phenotypes_twas(num_phenotypes, twas_dir, twas_name, filename_suffix, sep=",")
    
def analyze_random_phenotypes_twas_smultixcan(num_phenotypes, twas_dir, twas_name, filename_suffix):
    analyze_random_phenotypes_twas(num_phenotypes, twas_dir, twas_name, filename_suffix, sep="\t")